# The Weisfeiler-Lehman Isomorphism Test

The Weisfeiler-Lehman Isomorphism Test, also called WL-Test, is a classical result from the graph theory. It is an heuristic to find out if two graphs are isomorphic. The problem of the graph isomorphism does not have a definitive solution in polinomial time yet, making some people say it may be NP-Complete. The WL-Test offers one alternative that allows us to estimate it.

However, since this is an heuristic, the test is not perfect. Therefore, it fail in some simple cases and because of that new versions of this test were proposed.

In this notebook we will explore a little about the theory of the test and implement it in its classical form and also some newer versions.

In [3]:
!pip3 install networkx

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.1 MB/s eta 0:00:000m eta 0:00:010:00:01


In [1]:
import networkx as nx
def load_graph(file):
    G = nx.read_graphml(file).to_undirected()
    G.graph['phrase'] = G.graph.get('phrase', 'No phrase found')
    return G

# New implementation (gpt)

In [64]:
from __future__ import annotations
from pathlib import Path
from collections import defaultdict
from collections import Counter
import copy

# ---------- WL over trees ----------
def wl_tree_signature(G: nx.Graph) -> str:
    """
    1-WL color refinement on an (unlabeled) tree G.
    Returns a canonical signature string usable as a dict key.
    """
    # Relabel to 0..n-1 for array-friendly processing
    G = nx.relabel.convert_node_labels_to_integers(G, ordering="sorted")  # keeps attrs by default
    n = G.number_of_nodes()

    # adjacency as list of lists for speed
    adj = [list(G.neighbors(v)) for v in range(n)]  # neighbors() yields iterator.  [oai_citation:6‡networkx.org](https://networkx.org/documentation/stable/reference/classes/generated/networkx.Graph.neighbors.html?utm_source=chatgpt.com)
    colors = [0] * n
    tmp = [None] * n
    i = 0
    signature = ""
    while True:      
        hist = defaultdict(int)
        for c in colors:
            hist[c] += 1
        signature+= "|".join(f"{c}:{hist[c]}" for c in sorted(hist))
        signature += "#"
        for v in range(n):
            neigh_cols = sorted(colors[u] for u in adj[v])
            tmp[v] = str(colors[v]) +"_"+ str(tuple(neigh_cols))
        # compress tuples → small ints
        mapping = {}
        next_c = 0
        new_colors = [0] * n
        for v in range(n):
            key = tmp[v]
            #print(f"key is {key}")
            if key not in mapping:
                mapping[key] = next_c
                next_c += 1
            new_colors[v] = mapping[key]
        if new_colors == colors:
            break
        colors = new_colors
        i+=1

    return signature

In [45]:
mapping = {}
mapping[(0,0,0)] = 0
(0,0,0) in mapping

True

In [65]:
from pathlib import Path
GRAPH_DIR = "./UD_Spanish-GSD"
# --------- CONFIG ---------
FOLDER = Path(GRAPH_DIR)
OUT_CSV = Path("isomorphic_groups3.csv")

In [66]:
from tqdm import tqdm  # progress bar for long operations

groups = defaultdict(list)

for path in tqdm(FOLDER.rglob("*.graphml")):
    try:
         G = load_graph(path)
    except Exception as e:
        print(f"[WARN] Could not read {path}: {e}")
        continue

    
    sig = wl_tree_signature(G)
    #sig = wl_signature_hashed(G)
    #sig = str(fkWL(G, 1, verbose=False))
    groups[sig].append(str(path))

# --------- Write result ---------
with OUT_CSV.open("w", encoding="utf-8") as f:
    f.write("signature;count;files\n")
    for sig, files in groups.items():
        f.write(f"{sig};{len(files)};\"{'|'.join(files)}\"\n")

print(f"Done. Wrote {len(groups)} isomorphism classes to {OUT_CSV}")

14187it [00:07, 1779.14it/s]

Done. Wrote 12688 isomorphism classes to isomorphic_groups3.csv


In [68]:
import pandas as pd

df = pd.read_csv("isomorphic_groups3.csv", sep=";")
df


,signature,count,files
0,0:9#0:7|1:1|2:1#0:4|1:1|2:3|3:1#,5,UD_Spanish-GSD/es_gsd-ud-train_7825.graphml|UD...
1,0:19#0:13|1:1|2:3|3:2#0:5|1:1|2:3|3:1|4:5|5:1|...,1,UD_Spanish-GSD/es_gsd-ud-train_12948.graphml
2,0:24#0:15|1:3|2:3|3:1|4:2#0:3|1:1|2:5|3:1|4:1|...,1,UD_Spanish-GSD/es_gsd-ud-train_9745.graphml
3,0:37#0:24|1:1|2:4|3:5|4:2|5:1#0:10|1:1|2:4|3:6...,1,UD_Spanish-GSD/es_gsd-ud-train_3290.graphml
4,0:50#0:32|1:6|2:4|3:1|4:1|5:3|6:3#0:12|1:2|2:1...,1,UD_Spanish-GSD/es_gsd-ud-train_1588.graphml
...,...,...,...
12683,0:34#0:22|1:3|2:3|3:4|4:1|5:1#0:8|1:3|2:1|3:1|...,1,UD_Spanish-GSD/es_gsd-ud-train_7590.graphml
12684,0:29#0:19|1:3|2:2|3:2|4:2|5:1#0:3|1:6|2:2|3:1|...,1,UD_Spanish-GSD/es_gsd-ud-train_11797.graphml
12685,0:12#0:7|1:3|2:1|3:1#0:4|1:1|2:2|3:1|4:1|5:1|6...,1,UD_Spanish-GSD/es_gsd-ud-train_8663.graphml
12686,0:15#0:10|1:1|2:1|3:1|4:2#0:4|1:3|2:1|3:1|4:1|...,1,UD_Spanish-GSD/es_gsd-ud-train_1945.graphml


In [74]:
df[df["count"]==33]["signature"].values[0]

'0:13#0:8|1:2|2:2|3:1#0:2|1:4|2:1|3:2|4:1|5:1|6:1|7:1#0:1|1:4|2:1|3:1|4:1|5:1|6:1|7:1|8:1|9:1#'

In [75]:
df[df["signature"] == "0:13#0:8|1:2|2:2|3:1#0:2|1:4|2:1|3:2|4:1|5:1|6:1|7:1#0:1|1:4|2:1|3:1|4:1|5:1|6:1|7:1|8:1|9:1#"]["files"].values[0].split("|")

['UD_Spanish-GSD/es_gsd-ud-train_7835.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_909.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_8682.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_4727.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_8188.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_2360.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_13984.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_13446.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_562.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_13089.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_10222.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_7743.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_7510.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_8105.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_7604.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_7619.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_8647.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_6767.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_12780.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_7194.graphml',
 'UD_Spanish-GSD/es_gsd-ud-train_533.